In [2]:
%load_ext snakeviz

In [3]:
import rdflib
import rdflib_jsonld

import urllib.request
request_headers = {'Accept': 'application/rdf+xml'}
req = urllib.request.Request('http://id.nlm.nih.gov/mesh/D000900', headers = request_headers)
with urllib.request.urlopen(req) as response:
    page = response.read()
    

    graph = rdflib.Graph()

    graph.parse(data=page, format='xml')
    #predicate_query = graph.query("""select ?predicates where {?s ?predicates ?o}""")
    #for row in predicate_query:
    #    print('%s' % row)
    
    predicates = graph.predicates(subject=None, object=None)
    for predicate in predicates:
        print(predicate)

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#preferredConcept
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#dateCreated
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#dateRevised
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#concept
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#historyNote
http://id.nlm.nih.gov/mesh/vocab#broaderDescriptor
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#allowableQualifier
http://id.nlm.nih.gov/mesh/vocab#active
http://id.n

In [4]:
import rdflib
graph = rdflib.Graph()
graph.parse("mesh2019.nt", format='nt')

KeyboardInterrupt: 

In [3]:
import pandas as pd
df = pd.DataFrame(graph.predicates())
df = df.drop_duplicates()

In [10]:
df.to_hdf('graph_mesh', 'predicates')

In [8]:
from rdflib import Namespace
n = Namespace('http://id.nlm.nih.gov/mesh/')

In [ ]:
from rdflib.namespace import OWL, RDF, ClosedNamespace, RDFS

In [ ]:
vocabulary = rdflib.Graph().parse('ftp://nlmpubs.nlm.nih.gov/online/mesh/rdf/2019/vocabulary_1.0.0.ttl')
list_namespace = []
for node in vocabulary.all_nodes():
    label_node = g.label(node)
    if label_node != '':
        label_node = str(label_node)
        temp = label_node.split(' ')
        if len(temp) > 1:
            list_namespace.append(temp[1])
        else:
            list_namespace.append(label_node)

In [ ]:
len(list_namespace)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
sparql.setQuery("""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
    PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
    PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
    PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
    SELECT DISTINCT ?prop
    WHERE { ?resource ?prop ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

prop_res = []
for result in results["results"]["bindings"]:
    prop_res.append(result["prop"]["value"])

In [ ]:
def remove_prefix(prefix, string):
    res = string.partition(prefix)[2]
    return res

In [ ]:
import functools

prefixes = ['rdf-syntax-ns#', 'rdf-schema#', 'owl#', 'vocab#',]
properties = {}
for prefix in prefixes:
    remove_prefix_set = functools.partial(remove_prefix , prefix)
    filtered = set(map(remove_prefix_set, filter(lambda prop: prefix in prop, prop_res)))
    
    properties[prefix[:-1]] = filtered

In [ ]:
len(properties['vocab'])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")

for prefix in prefixes:
    prefix = prefix[:-1]
    for prop in properties[prefix]:

        sparql.setQuery(f' \
            PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
            PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
            PREFIX owl: <http://www.w3.org/2002/07/owl#> \
            PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> \
            PREFIX mesh: <http://id.nlm.nih.gov/mesh/> \
            PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/> \
            PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/> \
            PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/> \
            SELECT DISTINCT ?resource ?label \
            WHERE ' +"{" + f'?resource {prefix}:{prop} ?label' + "}")
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            print(result["resource"]["value"], result["label"]["value"])

        print(f'{prefix}:{prop}', len(results["results"]["bindings"]))
        print()

In [32]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
sparql.setQuery("""
    PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
    PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
    PREFIX owl: <http://www.w3.org/2002/07/owl#> \
    PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> \
    PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
    PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
    PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
    PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
    SELECT DISTINCT ?resource
    WHERE { ?resource ?prop ?value }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

resources = results["results"]["bindings"]

In [31]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
sparql.setQuery("""
    PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
    PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
    PREFIX owl: <http://www.w3.org/2002/07/owl#> \
    PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> \
    PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
    PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
    PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
    PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
    SELECT DISTINCT ?prop
    WHERE { ?resource ?prop ?value }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

properties = results["results"]["bindings"]

In [30]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
sparql.setQuery("""
    PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
    PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
    PREFIX owl: <http://www.w3.org/2002/07/owl#> \
    PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> \
    PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
    PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
    PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
    PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
    SELECT DISTINCT ?value
    WHERE { ?resource ?prop ?value }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

values = results["results"]["bindings"]

In [15]:
from neurolang import frontend as fe

/Users/gzanitti/Projects/INRIA/NeuroLang/neurolang/expression_pattern_matching.py:74: UserWarning: Warning in class SolverNonRecursiveDatalogNaive overwrites {'statement_intensional', 'fact'} from base DatalogBasic[typing.Any]
  warn(warn_message)


In [2]:
import pandas as pd
df = pd.read_hdf('graph_mesh', key='predicates')

In [5]:
df

,0
0,http://id.nlm.nih.gov/mesh/vocab#preferredTerm
1,http://id.nlm.nih.gov/mesh/vocab#dateCreated
2,http://www.w3.org/2000/01/rdf-schema#label
4,http://id.nlm.nih.gov/mesh/vocab#frequency
6,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
7,http://id.nlm.nih.gov/mesh/vocab#hasDescriptor
8,http://id.nlm.nih.gov/mesh/vocab#entryVersion
9,http://id.nlm.nih.gov/mesh/vocab#allowableQual...
10,http://id.nlm.nih.gov/mesh/vocab#preferredMapp...
13,http://id.nlm.nih.gov/mesh/vocab#source


In [5]:
dl = fe.NeurolangDL()

NameError: name 'fe' is not defined

In [63]:
@dl.add_symbol
def sparql_triplet_values(entity, property_, value):
    sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
    sparql.setQuery(f"""
        PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
        PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
        PREFIX owl: <http://www.w3.org/2002/07/owl#> \
        PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> 
        PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
        PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
        PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
        PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
        ASK 
        WHERE {{ {entity} {property_} "{value}" }}""")
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return len(results) > 0

In [58]:
def sparql_triplet(entity, property_):
    sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
    sparql.setQuery(f"""
        PREFIX rdf-syntax-ns: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \
        PREFIX rdf-schema: <http://www.w3.org/2000/01/rdf-schema#> \
        PREFIX owl: <http://www.w3.org/2002/07/owl#> \
        PREFIX vocab: <http://id.nlm.nih.gov/mesh/vocab#> 
        PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
        PREFIX mesh2019: <http://id.nlm.nih.gov/mesh/2019/>
        PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>
        PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
        ASK
        WHERE {{ {entity} {property_} ?value }}""")
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    return results

In [64]:
entity = '<http://id.nlm.nih.gov/mesh/D001921>'
property_ = 'rdf-schema:label'
res = sparql_triplet_values.value(entity, property_, 'Brain')
res

True

In [45]:
dl.add_tuple_set(
    ((e['value']['value'],) for e in values),
    name='values'
);
dl.add_tuple_set(
    ((e['prop']['value'],) for e in properties),
    name='properties'
);
dl.add_tuple_set(
    ((e['resource']['value'],) for e in resources),
    name='resources'
);

In [47]:
x = dl.new_symbol(name='x')
p = dl.new_symbol(name='p')
z = dl.new_symbol(name='z')
y = dl.new_symbol(name='y')

In [41]:
p[x] = dl.symbols.values[x]

In [42]:
dl.current_program

[p(x) ← values(x)]

In [65]:
dl.query(p[z], dl.symbols.resources[z] & sparql_triplet_values('<http://id.nlm.nih.gov/mesh/D001921>', 'rdf-schema:label',z))

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Line 8, column 71: Unresolved prefixed name: http:'

In [10]:
def generator_file():
    filepath = 'mesh2019.nt'
    with open(filepath) as fp:
        for count, line in enumerate(fp.readlines()):
            yield line[:-3].split(' ')
            if count > 100000: break
            
import pandas as pd
data = generator_file()
df = pd.DataFrame(data)

In [11]:
df.rename(columns={0: 'object', 1: 'predicate', 2: 'value'}, inplace=True)

In [12]:
from neurolang import frontend as fe
dl = fe.NeurolangDL()

dl.add_tuple_set(
    ((e,) for e in df['value'].drop_duplicates()),
    name='values'
);
dl.add_tuple_set(
    ((e,) for e in df['predicate'].drop_duplicates()),
    name='predicates'
);
dl.add_tuple_set(
    ((e,) for e in df['object'].drop_duplicates()),
    name='objects'
);

dl.add_tuple_set(
    ((e['object'], e['predicate'], e['value'],) for _, e in df.iterrows()),
    name='domain'
);

In [13]:
[tuple(f[:3]) for f in df.iloc[:10].values]

[('<http://id.nlm.nih.gov/mesh/2019/A01.111>',
  '<http://id.nlm.nih.gov/mesh/vocab#parentTreeNumber>',
  '<http://id.nlm.nih.gov/mesh/2019/A01>'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.111>',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://id.nlm.nih.gov/mesh/vocab#TreeNumber>'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.111>',
  '<http://www.w3.org/2000/01/rdf-schema#label>',
  '"A01.111"@en'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.236.249>',
  '<http://id.nlm.nih.gov/mesh/vocab#parentTreeNumber>',
  '<http://id.nlm.nih.gov/mesh/2019/A01.236>'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.236.249>',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://id.nlm.nih.gov/mesh/vocab#TreeNumber>'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.236.249>',
  '<http://www.w3.org/2000/01/rdf-schema#label>',
  '"A01.236.249"@en'),
 ('<http://id.nlm.nih.gov/mesh/2019/A01.236.500>',
  '<http://id.nlm.nih.gov/mesh/vocab#parentTreeNumber>',
  '<http://id.nlm.nih.gov/mesh/2019/

In [14]:
o = dl.new_symbol(name='o')
p = dl.new_symbol(name='p')
v = dl.new_symbol(name='v')
RDF = dl.new_symbol(name='RDF')

values = dl.new_symbol(name='values')

query = dl.query(RDF[o,p,v], dl.symbols.domain(o,p,v))

In [19]:
%%snakeviz
res = dl.query(values(v), RDF('<http://id.nlm.nih.gov/mesh/2019/A01.236.249>', p, v))
for e in res:
    print(e)

('<http://id.nlm.nih.gov/mesh/2019/A01.236>',)
('<http://id.nlm.nih.gov/mesh/vocab#TreeNumber>',)
('"A01.236.249"@en',)
 
*** Profile stats marshalled to file '/var/folders/dp/jss5gg557c78z1jk8d9knnbc0000gn/T/tmpjudr8mro'. 
Embedding SnakeViz in this document...


In [18]:
type(dl.symbols.domain.expression.value)

neurolang.solver_datalog_naive.WrappedRelationalAlgebraSet

In [185]:
df.loc[df['object'] == '<http://id.nlm.nih.gov/mesh/2019/A01.236.249>']

,object,predicate,value,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
3,<http://id.nlm.nih.gov/mesh/2019/A01.236.249>,<http://id.nlm.nih.gov/mesh/vocab#parentTreeNu...,<http://id.nlm.nih.gov/mesh/2019/A01.236>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,<http://id.nlm.nih.gov/mesh/2019/A01.236.249>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://id.nlm.nih.gov/mesh/vocab#TreeNumber>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,<http://id.nlm.nih.gov/mesh/2019/A01.236.249>,<http://www.w3.org/2000/01/rdf-schema#label>,"""A01.236.249""@en",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [186]:
s = dl.new_symbol(name='s')
res = dl.query(values(o), 
               RDF(o, '<http://id.nlm.nih.gov/mesh/vocab#parentTreeNumber>', '<http://id.nlm.nih.gov/mesh/2019/A01.378.610>'))
for e in res:
    print(e)

('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.100>',)
('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.250>',)
('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.400>',)
('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.450>',)
('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.500>',)
('<http://id.nlm.nih.gov/mesh/2019/A01.378.610.750>',)


In [187]:
s = dl.new_symbol(name='s')
res = dl.query(values(o, p), 
               RDF(o, p, '"5-S-lipoylhydroxytyrosol"@en'))
for e in res:
    print(e)

('<http://id.nlm.nih.gov/mesh/2019/C000589398>', '<http://www.w3.org/2000/01/rdf-schema#label>')


In [21]:
import rdflib
graph = rdflib.Graph()
graph.parse("human_brain_ontoly.xml", format='xml')

TypeError: sequence item 0: expected str instance, NoneType found